# Integrating the Sagemaker endpoint into our movie recommender web application

In its current form the K-NN model which we deployed in our previous lab to get recommended movies for a user works nicely from a Jupyter notebook. However it is hard to integrate into a business application. It requires a user embedding vector as input and returns a JSON output which is specific to our model (e.g. movie distances) and contains information which is not relevant to our application.

In the next step we will write a simple lambda function which takes a user id as input and returns a simple list of recommended movies in JSON format. The lambda function will be responsible to transform the user id into a user embedding, invoke the sagemaker endpoint and return a simplified response containing only movie ids to the user. The lambda functions will be fronted by an API Gateway, to provide the REST endpoint. See following figure a simple architecture:

![arch](images/architecture-lab2.png)

Adding this integration layer creates a simple REST-API based contract between our business application and our machine learning backend functionality, which has following benefits:

- It allows us to change the deployed machine learning model in the backend without requiring any code changes to our business application
- Allows to easily split the work/responsibilities into separate teams (e.g. bussiness app development and data science team)

# Developing and deploying the lambda function

For deployment of the lambda function we will use the [serverless framework](https://serverless.com/). Let's start by installing the serverless framework and required dependencies.

In [ ]:
!npm install -g serverless serverless-iam-roles-per-function

All required files for deployment are available in the integrationLambda folder. 
The file [serverless.yml](./integrationLambda/serverless.yml) contains all configuration required for deployment, e.g. lambda function properties, permissions or environment variables.
The file [handler.py](./integrationLambda/handler.py) cotnains the code of the lambda function.

Feel free to explore the code now in detail if you wish. 

Next we need to set a few environment variables, so the lambda knows which sagemaker endpoint to call and where user embeddings are stored.
In serverless.yml update following two properties:

```
EMBEDDINGS_S3_PATH: <insert the path to the embeddings file show at the end of the previous lab notebook>
SAGEMAKER_ENDPOINT_NAME: <insert the name of the sagemaker endpoint also shown at the end of the previous notebook>
```

here is an example:

```
EMBEDDINGS_S3_PATH: s3://sagemaker-assets-jlanger/fm/output/factorization-machines-2019-10-24-19-30-22-215/output/user_embeddings.pickle
SAGEMAKER_ENDPOINT_NAME: knn-2019-10-24-19-35-08-799
```

Now let's go ahead and deploy the lambda function

In [5]:
!cd integration_lambda; sls deploy

Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: Uploading CloudFormation file to S3...
Serverless: Uploading artifacts...
Serverless: Uploading service recommendationendpointintegration.zip file to S3 (3.76 KB)...
Serverless: Validating template...
Serverless: Updating Stack...
Serverless: Checking Stack update progress...
..............
Serverless: Stack update finished...
Service Information
service: recommendationendpointintegration
stage: dev
region: us-east-1
stack: recommendationendpointintegration-dev
resources: 11
api keys:
  None
endpoints:
  GET - https://sbehr8v2h0.execute-api.us-east-1.amazonaws.com/dev/recommendEndpoint
functions:
  hello: recommendationendpointintegration-dev-hello
layers:
  None
Serverless: Run the "serverless" command to setup monitoring, troubleshooting and testing.


## Validating the endpoint

Now check that the endpoint returns valid results. You can find the url in the output of the last command under endpoints. Make sure to attach a URL parameter to pass in the requested user_id.
The URL should have following format:

https://\<unique_id>.execute-api.us-east-1.amazonaws.com/dev/recommendEndpoint?user_id=\<user_id>

Here is an example:

https://8c7g5h0jv7.execute-api.us-east-1.amazonaws.com/dev/recommendEndpoint?user_id=3



The request should return a response similar to this (movie ID's can vary):

```
{"itemList": [{"itemId": "494"}, {"itemId": "1656"}, {"itemId": "1125"}, {"itemId": "652"}, {"itemId": "814"}, {"itemId": "1130"}, {"itemId": "137"}, {"itemId": "1525"}, {"itemId": "851"}, {"itemId": "1467"}, {"itemId": "1235"}, {"itemId": "185"}, {"itemId": "64"}, {"itemId": "498"}, {"itemId": "191"}, {"itemId": "1599"}, {"itemId": "896"}, {"itemId": "168"}, {"itemId": "1604"}, {"itemId": "647"}, {"itemId": "1194"}, {"itemId": "198"}, {"itemId": "478"}, {"itemId": "1616"}, {"itemId": "1662"}, {"itemId": "709"}, {"itemId": "1536"}, {"itemId": "514"}, {"itemId": "1650"}, {"itemId": "223"}, {"itemId": "529"}, ....]}
```

**Please note down the endpoint URL shown in the output from the sls deploy command. As we will need it later**

e.g. https://sbehr8v2h0.execute-api.us-east-1.amazonaws.com/dev/recommendEndpoint

__[now jump back into the original Lab Guidebook - Deploying the integration lambda](https://github.com/johanneslanger/recommendations-on-aws-workshop/tree/master/lab-2-recommendations-with-sagemaker#integration-the-endpoint-into-the-movie-recommender-app)__